In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Dataset from Kaggle (https://www.kaggle.com/datasets/urvishahir/electric-vehicle-specifications-dataset-2025)

In [2]:
import sqlite3
import pandas as pd
import os

# Specify the CSV file path and database name
csv_file = os.path.join('data', 'electric_vehicles_spec_2025.csv.csv')  # Replace 'your_file.csv' with your actual file name
db_file = 'my_database.db'

# Read CSV into DataFrame
df = pd.read_csv(csv_file)

# Create SQLite database and write DataFrame to it
conn = sqlite3.connect(db_file)
df.to_sql('my_table', conn, if_exists='replace', index=False)

478

In [3]:
# Test the connection by querying the first 5 rows
test_df = pd.read_sql_query("SELECT * FROM my_table LIMIT 5;", conn)
print(test_df)

    brand                model  top_speed_kmh  battery_capacity_kWh  \
0  Abarth     500e Convertible            155                  37.8   
1  Abarth       500e Hatchback            155                  37.8   
2  Abarth  600e Scorpionissima            200                  50.8   
3  Abarth         600e Turismo            200                  50.8   
4  Aiways                   U5            150                  60.0   

  battery_type  number_of_cells  torque_nm  efficiency_wh_per_km  range_km  \
0  Lithium-ion            192.0      235.0                   156       225   
1  Lithium-ion            192.0      235.0                   149       225   
2  Lithium-ion            102.0      345.0                   158       280   
3  Lithium-ion            102.0      345.0                   158       280   
4  Lithium-ion              NaN      310.0                   156       315   

   acceleration_0_100_s  ...  towing_capacity_kg cargo_volume_l  seats  \
0                   7.0  ...  

In [4]:
test_df = pd.read_sql_query("SELECT model FROM my_table where top_speed_kmh > 210;", conn)
print(test_df)

                             model
0                  S6 Avant e-tron
1              S6 Sportback e-tron
2                       SQ6 e-tron
3             SQ6 e-tron Sportback
4                     e-tron GT RS
..                             ...
67  Model 3 Performance (Highland)
68              Model S Dual Motor
69                   Model S Plaid
70              Model X Dual Motor
71                   Model X Plaid

[72 rows x 1 columns]


In [5]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')
from langchain.agents import AgentType
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain.chat_models import init_chat_model
import sqlalchemy


llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai", temperature=0.0)

db_file = 'my_database.db'
engine = sqlalchemy.create_engine(f"sqlite:///{db_file}")

db = SQLDatabase(engine)

agent_executor = create_sql_agent(llm=llm, db=db, verbose=True, handle_parsing_errors=True, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

In [6]:
agent_executor.invoke("What are the top 5 electric vehicles with a top speed greater than 210 km/h?")  # Example query



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:my_tableAction: sql_db_schema
Action Input: my_table
CREATE TABLE my_table (
	brand TEXT, 
	model TEXT, 
	top_speed_kmh INTEGER, 
	"battery_capacity_kWh" REAL, 
	battery_type TEXT, 
	number_of_cells REAL, 
	torque_nm REAL, 
	efficiency_wh_per_km INTEGER, 
	range_km INTEGER, 
	acceleration_0_100_s REAL, 
	fast_charging_power_kw_dc REAL, 
	fast_charge_port TEXT, 
	towing_capacity_kg REAL, 
	cargo_volume_l TEXT, 
	seats INTEGER, 
	drivetrain TEXT, 
	segment TEXT, 
	length_mm INTEGER, 
	width_mm INTEGER, 
	height_mm INTEGER, 
	car_body_type TEXT, 
	source_url TEXT
)

/*
3 rows from my_table table:
brand	model	top_speed_kmh	battery_capacity_kWh	battery_type	number_of_cells	torque_nm	efficiency_wh_per_km	range_km	acceleration_0_100_s	fast_charging_power_kw_dc	fast_charge_port	towing_capacity_kg	cargo_volume_l	seats	drivetrain	segment	length_mm	width_mm	height_mm	car_body_type	source_url
Abarth	500e Convertib

{'input': 'What are the top 5 electric vehicles with a top speed greater than 210 km/h?',
 'output': 'The top 5 electric vehicles with a top speed greater than 210 km/h are:\n* Maserati GranTurismo Folgore (325 km/h)\n* Porsche Taycan Turbo GT Weissach (305 km/h)\n* Maserati GranCabrio Folgore (290 km/h)\n* Porsche Taycan Turbo GT (290 km/h)\n* Tesla Model S Plaid (282 km/h)'}

In [7]:
result = agent_executor.invoke("What are the top 5 electric vehicles with a top speed greater than 210 km/h?")  # Example query



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:my_tableAction: sql_db_schema
Action Input: my_table
CREATE TABLE my_table (
	brand TEXT, 
	model TEXT, 
	top_speed_kmh INTEGER, 
	"battery_capacity_kWh" REAL, 
	battery_type TEXT, 
	number_of_cells REAL, 
	torque_nm REAL, 
	efficiency_wh_per_km INTEGER, 
	range_km INTEGER, 
	acceleration_0_100_s REAL, 
	fast_charging_power_kw_dc REAL, 
	fast_charge_port TEXT, 
	towing_capacity_kg REAL, 
	cargo_volume_l TEXT, 
	seats INTEGER, 
	drivetrain TEXT, 
	segment TEXT, 
	length_mm INTEGER, 
	width_mm INTEGER, 
	height_mm INTEGER, 
	car_body_type TEXT, 
	source_url TEXT
)

/*
3 rows from my_table table:
brand	model	top_speed_kmh	battery_capacity_kWh	battery_type	number_of_cells	torque_nm	efficiency_wh_per_km	range_km	acceleration_0_100_s	fast_charging_power_kw_dc	fast_charge_port	towing_capacity_kg	cargo_volume_l	seats	drivetrain	segment	length_mm	width_mm	height_mm	car_body_type	source_url
Abarth	500e Convertib

In [11]:
print(result['output'])

The top 5 electric vehicles with a top speed greater than 210 km/h are:
* Maserati GranTurismo Folgore (325 km/h)
* Porsche Taycan Turbo GT Weissach (305 km/h)
* Maserati GranCabrio Folgore (290 km/h)
* Porsche Taycan Turbo GT (290 km/h)
* Tesla Model S Plaid (282 km/h)


In [13]:
result = agent_executor.invoke("What models does Audi offer? Group by car type!", handle_parsing_errors=True)  # Example query



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:my_tableAction: sql_db_schema
Action Input: my_table
CREATE TABLE my_table (
	brand TEXT, 
	model TEXT, 
	top_speed_kmh INTEGER, 
	"battery_capacity_kWh" REAL, 
	battery_type TEXT, 
	number_of_cells REAL, 
	torque_nm REAL, 
	efficiency_wh_per_km INTEGER, 
	range_km INTEGER, 
	acceleration_0_100_s REAL, 
	fast_charging_power_kw_dc REAL, 
	fast_charge_port TEXT, 
	towing_capacity_kg REAL, 
	cargo_volume_l TEXT, 
	seats INTEGER, 
	drivetrain TEXT, 
	segment TEXT, 
	length_mm INTEGER, 
	width_mm INTEGER, 
	height_mm INTEGER, 
	car_body_type TEXT, 
	source_url TEXT
)

/*
3 rows from my_table table:
brand	model	top_speed_kmh	battery_capacity_kWh	battery_type	number_of_cells	torque_nm	efficiency_wh_per_km	range_km	acceleration_0_100_s	fast_charging_power_kw_dc	fast_charge_port	towing_capacity_kg	cargo_volume_l	seats	drivetrain	segment	length_mm	width_mm	height_mm	car_body_type	source_url
Abarth	500e Convertib

In [14]:
print(result['output'])

Audi offers the following models grouped by car type:
Liftback Sedan: A6 Sportback e-tron, A6 Sportback e-tron performance, A6 Sportback e-tron quattro, S6 Sportback e-tron
SUV: Q4 Sportback e-tron 40, Q4 Sportback e-tron 45, Q4 Sportback e-tron 45 quattro, Q4 Sportback e-tron 55 quattro, Q4 e-tron 40, Q4 e-tron 45
